In [ ]:
import diffusers
import torch

from pipeline import FTEditFluxPipeline
from processor.ft_editing_attn_processor import FluxAttentionReplace, P2PFlux_JointAttnProcessor2_0

diffusers.utils.logging.set_verbosity_error()

pipe = FTEditFluxPipeline.from_pretrained("black-forest-labs/FLUX.1-dev", torch_dtype=torch.bfloat16)
pipe.to("cuda")

source_img = "assets/sources/coffee.jpg"
source_prompt = "a cup of coffee with a drawing of a tulip put on the wooden table."
target_prompt = "a cup of coffee with a drawing of a lion put on the wooden table."
prompts = [source_prompt, target_prompt]

num_inference_steps = 25
controller = FluxAttentionReplace(
    prompts=prompts,
    num_steps=num_inference_steps,
    attn_ratio=0.15,
    num_att_layers=37,
)
pipe.add_processor(
    after_layer=0,
    before_layer=36,
    filter_name="transformer_blocks",
    target_processor=P2PFlux_JointAttnProcessor2_0,
    controller=controller,  # kwargs for `P2PFlux_JointAttnProcessor2_0`
)

image = pipe(
    source_img,
    source_prompt,
    target_prompt,
    guidance_scale=2,
    ly_ratio=1.0,
    num_inference_steps=num_inference_steps,
).images[0]
image.save("assets/results/coffee_ftedit.jpg")
image